In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sns
import DbConnection as dbcon
%matplotlib inline

In [5]:
import os
os.listdir('data')

['COVID19_open_line_list.csv',
 '.DS_Store',
 'covid_19_data.csv',
 'covid19_xmlToCSV.csv',
 'covid19_date_stats_n.csv',
 'COVID19_open_line_list_file_cleaned.csv',
 'web_data_covid19.csv',
 'Cleaned_Tweet_data.csv',
 'twitter_data_1.txt']

In [6]:
data_csv=pd.read_csv('data/COVID19_open_line_list_file_cleaned.csv')
data_csv.head()

,ID,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,date_confirmation,source,admin2,admin1,country_new,admin_id
0,1.0,"Chaohu City, Hefei City",Anhui,China,1.0,31.646960,117.716600,admin3,22.01.2020,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,Hefei City,Anhui,China,340181
1,2.0,"Baohe District, Hefei City",Anhui,China,1.0,31.778630,117.331900,admin3,23.01.2020,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,Hefei City,Anhui,China,340111
2,3.0,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,23.01.2020,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,Hefei City,Anhui,China,340104
3,4.0,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,23.01.2020,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,Hefei City,Anhui,China,340104
4,5.0,"Feidong County, Hefei City",Anhui,China,1.0,32.001230,117.568100,admin3,23.01.2020,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,Hefei City,Anhui,China,340122


## Pre proecessing the data

In [7]:
data_xml=pd.read_csv('data/covid19_xmlToCSV.csv')
data_xml.head()

,Unnamed: 0,_id,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,0,5e9ed0b93b43ce83895586a8,10/04/2020,10,4,2020,61,1,Afghanistan,AF,AFG,37172386
1,1,5e9ed0b93b43ce83895586a9,09/04/2020,9,4,2020,56,3,Afghanistan,AF,AFG,37172386
2,2,5e9ed0b93b43ce83895586aa,08/04/2020,8,4,2020,30,4,Afghanistan,AF,AFG,37172386
3,3,5e9ed0b93b43ce83895586ab,07/04/2020,7,4,2020,38,0,Afghanistan,AF,AFG,37172386
4,4,5e9ed0b93b43ce83895586ac,06/04/2020,6,4,2020,29,2,Afghanistan,AF,AFG,37172386


In [8]:
data_xml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               19538 non-null  int64 
 1   _id                      19538 non-null  object
 2   dateRep                  19538 non-null  object
 3   day                      19538 non-null  int64 
 4   month                    19538 non-null  int64 
 5   year                     19538 non-null  int64 
 6   cases                    19538 non-null  int64 
 7   deaths                   19538 non-null  int64 
 8   countriesAndTerritories  19538 non-null  object
 9   geoId                    19484 non-null  object
 10  countryterritoryCode     19410 non-null  object
 11  popData2018              19538 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 1.8+ MB


In [9]:
data_xml.dropna(inplace=True)

In [10]:
data_xml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19356 entries, 0 to 19537
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               19356 non-null  int64 
 1   _id                      19356 non-null  object
 2   dateRep                  19356 non-null  object
 3   day                      19356 non-null  int64 
 4   month                    19356 non-null  int64 
 5   year                     19356 non-null  int64 
 6   cases                    19356 non-null  int64 
 7   deaths                   19356 non-null  int64 
 8   countriesAndTerritories  19356 non-null  object
 9   geoId                    19356 non-null  object
 10  countryterritoryCode     19356 non-null  object
 11  popData2018              19356 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 1.9+ MB


In [13]:
## Take important column
data_xml_2=data_xml.loc[:,['countriesAndTerritories','cases','deaths']]
data_xml_2.head()

,countriesAndTerritories,cases,deaths
0,Afghanistan,61,1
1,Afghanistan,56,3
2,Afghanistan,30,4
3,Afghanistan,38,0
4,Afghanistan,29,2


In [14]:
data_xml_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19356 entries, 0 to 19537
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   countriesAndTerritories  19356 non-null  object
 1   cases                    19356 non-null  int64 
 2   deaths                   19356 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 604.9+ KB


In [89]:
## Group wise country and cases and death

In [15]:
data_xml_3=data_xml_2.groupby(['countriesAndTerritories'],as_index=False)['cases','deaths'].agg('sum')

/Users/rahul/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [16]:
data_xml_3.head()

,countriesAndTerritories,cases,deaths
0,Afghanistan,968,30
1,Albania,818,46
2,Algeria,3332,470
3,Andorra,1166,50
4,Angola,38,4


In [17]:
data_xml_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   countriesAndTerritories  199 non-null    object
 1   cases                    199 non-null    int64 
 2   deaths                   199 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.2+ KB


## Create a table in postgre for storing xml file

In [18]:
table_name='covid19_xml_tbl'

In [19]:
column_name=['countriesandterritories VARCHAR(100) PRIMARY KEY','cases numeric','deaths numeric']


In [ ]:
dbcon.create_table(table_name,column_name,dbcon.
                   get_dbCon())


## Insert data from data_xml_3 dataframe to covid19_xml_tbl

In [ ]:
db_column=['countriesandterritories','cases','deaths']
dbcon.insert_csv_table(data_xml_3,table_name,db_column)

In [20]:
## CHECK the insetion part
dbcon.select_count(table_name,dbcon.get_dbCon())

[(199,)]


## Pre-Process the web scrapping and store into postgre

In [21]:
data_web=pd.read_csv('data/web_data_covid19.csv')
data_web.head()

,Country,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Serious_Critical
0,USA,"782,987","41,777","71,770","669,440","13,576"
1,Spain,"200,210","20,852","80,587","98,771","7,371"
2,Italy,"181,228","24,114","48,877","108,237","2,573"
3,France,"155,383","20,265","37,409","97,709","5,683"
4,Germany,"146,398","4,706","91,500","50,192","2,889"


In [22]:
data_web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country           212 non-null    object
 1   TotalCases        212 non-null    object
 2   TotalDeaths       212 non-null    object
 3   TotalRecovered    199 non-null    object
 4   ActiveCases       212 non-null    object
 5   Serious_Critical  137 non-null    object
dtypes: object(6)
memory usage: 10.1+ KB


In [23]:
# drop the null variable because I want to analyze data
#data_web.dropna(inplace=True,subset=['ActiveCases'])
data_web.dropna(inplace=True)

In [24]:
data_web.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 211
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country           136 non-null    object
 1   TotalCases        136 non-null    object
 2   TotalDeaths       136 non-null    object
 3   TotalRecovered    136 non-null    object
 4   ActiveCases       136 non-null    object
 5   Serious_Critical  136 non-null    object
dtypes: object(6)
memory usage: 7.4+ KB


In [25]:
data_web.head()

,Country,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Serious_Critical
0,USA,"782,987","41,777","71,770","669,440","13,576"
1,Spain,"200,210","20,852","80,587","98,771","7,371"
2,Italy,"181,228","24,114","48,877","108,237","2,573"
3,France,"155,383","20,265","37,409","97,709","5,683"
4,Germany,"146,398","4,706","91,500","50,192","2,889"


### Preprocessing web data 

In [26]:
data_web.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 211
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country           136 non-null    object
 1   TotalCases        136 non-null    object
 2   TotalDeaths       136 non-null    object
 3   TotalRecovered    136 non-null    object
 4   ActiveCases       136 non-null    object
 5   Serious_Critical  136 non-null    object
dtypes: object(6)
memory usage: 7.4+ KB


In [104]:
## Remove comma
## replace empty with 0

In [27]:
data_web['TotalCases']=data_web['TotalCases'].str.replace(",","")
data_web['TotalCases']=data_web['TotalCases'].str.replace(" ",'0')

In [28]:
data_web['TotalRecovered']=data_web['TotalRecovered'].str.replace(",","")
data_web['TotalRecovered']=data_web['TotalRecovered'].str.replace(" ","0")

In [29]:
data_web['TotalDeaths']=data_web['TotalDeaths'].str.replace(",","")
data_web['TotalDeaths']=data_web['TotalDeaths'].str.replace(" ","0")

In [30]:
data_web['ActiveCases']=data_web['ActiveCases'].str.replace(",","")
data_web['ActiveCases']=data_web['ActiveCases'].str.replace(" ","0")

In [31]:
data_web['Serious_Critical']=data_web['Serious_Critical'].str.replace(",","")
data_web['Serious_Critical']=data_web['Serious_Critical'].str.replace(" ","0")

In [32]:
data_web.head()

,Country,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Serious_Critical
0,USA,782987,41777,71770,669440,13576
1,Spain,200210,20852,80587,98771,7371
2,Italy,181228,24114,48877,108237,2573
3,France,155383,20265,37409,97709,5683
4,Germany,146398,4706,91500,50192,2889


In [33]:
data_web.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 211
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country           136 non-null    object
 1   TotalCases        136 non-null    object
 2   TotalDeaths       136 non-null    object
 3   TotalRecovered    136 non-null    object
 4   ActiveCases       136 non-null    object
 5   Serious_Critical  136 non-null    object
dtypes: object(6)
memory usage: 7.4+ KB


In [34]:
data_web['TotalCases']=data_web['TotalCases'].astype('float64')

In [35]:
data_web['TotalDeaths']=data_web['TotalDeaths'].astype('float64')

In [36]:
data_web['TotalRecovered']=data_web['TotalRecovered'].astype('float64')

In [37]:
data_web['ActiveCases']=data_web['ActiveCases'].astype('float64')


In [38]:
data_web['Serious_Critical']=data_web['Serious_Critical'].astype('float64')

In [39]:
table_name='covid19_web_tbl'

In [40]:
column_name=['country VARCHAR(100)','total_case numeric','total_death VARCHAR(100)','total_recovered string',
             'active_cases numeric','serious_critical numeric']

In [ ]:
dbcon.create_table(table_name,column_name,dbcon.get_dbCon())

### Insertion of web data(data_web) into postgres (covid19_web_tbl)

In [41]:
db_col=['country','total_case','total_death','total_recovered',
             'active_cases','serious_critical']

In [ ]:
dbcon.insert_csv_table(data_web,table_name,db_col)

In [42]:
## Check the insertion
dbcon.select_count(table_name,dbcon.get_dbCon())

[(136,)]


# Relational query INNER JOIN on three tables -covid19_xml_tbl,covid19_web_tbl,country_wuhan_visit_tbl

In [43]:
query="""
SELECT covid19_xml_tbl.countriesandterritories,
covid19_xml_tbl.cases,
country_wuhan_visit_tbl.total_wuhan_visited,
covid19_web_tbl.total_recovered,
covid19_web_tbl.active_cases
FROM covid19_xml_tbl
 INNER JOIN covid19_web_tbl ON covid19_xml_tbl.countriesandterritories = covid19_web_tbl.country
 INNER JOIN country_wuhan_visit_tbl ON country_wuhan_visit_tbl.country= covid19_web_tbl.country"""





In [44]:
all_data_df=dbcon.select_res_dataframe(query)

Query run successful
PostgreSQL connection is closed


In [45]:
all_data_df.head()

,countriesandterritories,cases,total_wuhan_visited,total_recovered,active_cases
0,Spain,304892.0,13.0,80587.0,98771.0
1,Italy,287252.0,588.0,48877.0,108237.0
2,France,172668.0,19.0,37409.0,97709.0
3,Germany,227050.0,20.0,91500.0,50192.0
4,Iran,132440.0,28.0,59273.0,19023.0


In [46]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   countriesandterritories  39 non-null     object 
 1   cases                    39 non-null     float64
 2   total_wuhan_visited      39 non-null     float64
 3   total_recovered          39 non-null     float64
 4   active_cases             39 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.6+ KB


## Insert fetched data from db to    csv file

In [47]:
filename=os.getcwd()+"all_file_covid_data.csv"

In [48]:
filename

'/Users/rahul/Documents/NCI/PROJECT/DAP/covid_19_group_3_code_19173105_19179391_18185975_19198400/Rahul_json_csvall_file_covid_data.csv'

In [49]:
df_all_n=dbcon.insert_df_into_csv(all_data_df,filename)

Data inserted successfull to the csv file: /Users/rahul/Documents/NCI/PROJECT/DAP/covid_19_group_3_code_19173105_19179391_18185975_19198400/Rahul_json_csvall_file_covid_data.csv


In [50]:
df_all_n.head()

,countriesandterritories,cases,total_wuhan_visited,total_recovered,active_cases
0,Spain,304892.0,13.0,80587.0,98771.0
1,Italy,287252.0,588.0,48877.0,108237.0
2,France,172668.0,19.0,37409.0,97709.0
3,Germany,227050.0,20.0,91500.0,50192.0
4,Iran,132440.0,28.0,59273.0,19023.0


## TWITTER data manipulation

##  Save twitter json  data from postgres table to csv file

In [51]:
table_name='tweet_json_tbl'
query="SELECT * FROM "+ table_name

In [52]:
df_json_twitter=dbcon.select_res_dataframe(query)

Query run successful
PostgreSQL connection is closed


In [53]:
df_json_twitter.head()

,id,text,user_name,created_at,location,timestamp_ms,quote_count,reply_count,retweet_count,favorite_count,hashtags
0,1.250014e+18,RT @eileeniorio: Talking to the @BBCNews @Bil...,ChrisJGodlewski,Tue Apr 14 10:53:12 +0000 2020,"Alsace, France",1.586862e+12,0.0,0.0,0.0,0.0,COVID19
1,1.250014e+18,My tribute for today. Congratulations on exec...,bearhugz25,Tue Apr 14 10:53:12 +0000 2020,Philippines,1.586862e+12,0.0,0.0,0.0,0.0,None
2,1.250014e+18,RT @covid19esp_: 🗞☣ #COVID19 14/4 - 12h\n\n☣ 1...,margarjimen,Tue Apr 14 10:53:12 +0000 2020,None,1.586862e+12,0.0,0.0,0.0,0.0,COVID19
3,1.250014e+18,RT @Brevesdepresse: ⚡🇲🇫FLASH - Considérés comm...,Reazy92i,Tue Apr 14 10:53:12 +0000 2020,"Poitiers, France",1.586862e+12,0.0,0.0,0.0,0.0,infirmiers
4,1.250014e+18,Ok @opodo I'm now at the end of my tether. Had...,Lesley_Allen_,Tue Apr 14 10:53:12 +0000 2020,Bangor Co Down,1.586862e+12,0.0,0.0,0.0,0.0,None


In [54]:
df_json_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   float64
 1   text            1000 non-null   object 
 2   user_name       1000 non-null   object 
 3   created_at      1000 non-null   object 
 4   location        652 non-null    object 
 5   timestamp_ms    1000 non-null   float64
 6   quote_count     1000 non-null   float64
 7   reply_count     1000 non-null   float64
 8   retweet_count   1000 non-null   float64
 9   favorite_count  1000 non-null   float64
 10  hashtags        495 non-null    object 
dtypes: float64(6), object(5)
memory usage: 86.1+ KB


In [55]:
df_json_twitter.dropna(subset=['location'],inplace=True)

In [56]:
df_json_twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 652 entries, 0 to 999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              652 non-null    float64
 1   text            652 non-null    object 
 2   user_name       652 non-null    object 
 3   created_at      652 non-null    object 
 4   location        652 non-null    object 
 5   timestamp_ms    652 non-null    float64
 6   quote_count     652 non-null    float64
 7   reply_count     652 non-null    float64
 8   retweet_count   652 non-null    float64
 9   favorite_count  652 non-null    float64
 10  hashtags        336 non-null    object 
dtypes: float64(6), object(5)
memory usage: 61.1+ KB


In [57]:
file_name='/tweet_json.csv'
filpath=os.getcwd()+file_name

In [58]:
df_tweet_new=dbcon.insert_df_into_csv(df_json_twitter,filpath)

Data inserted successfull to the csv file: /Users/rahul/Documents/NCI/PROJECT/DAP/covid_19_group_3_code_19173105_19179391_18185975_19198400/Rahul_json_csv/tweet_json.csv


In [59]:
df_tweet_new.head()

,id,text,user_name,created_at,location,timestamp_ms,quote_count,reply_count,retweet_count,favorite_count,hashtags
0,1.250014e+18,RT @eileeniorio: Talking to the @BBCNews @Bil...,ChrisJGodlewski,Tue Apr 14 10:53:12 +0000 2020,"Alsace, France",1.586862e+12,0.0,0.0,0.0,0.0,COVID19
1,1.250014e+18,My tribute for today. Congratulations on exec...,bearhugz25,Tue Apr 14 10:53:12 +0000 2020,Philippines,1.586862e+12,0.0,0.0,0.0,0.0,NaN
2,1.250014e+18,RT @Brevesdepresse: ⚡🇲🇫FLASH - Considérés comm...,Reazy92i,Tue Apr 14 10:53:12 +0000 2020,"Poitiers, France",1.586862e+12,0.0,0.0,0.0,0.0,infirmiers
3,1.250014e+18,Ok @opodo I'm now at the end of my tether. Had...,Lesley_Allen_,Tue Apr 14 10:53:12 +0000 2020,Bangor Co Down,1.586862e+12,0.0,0.0,0.0,0.0,NaN
4,1.250014e+18,RT @PresidenciaSV: Este es el Decreto Ejecutiv...,tecmel1,Tue Apr 14 10:53:12 +0000 2020,el salvador,1.586862e+12,0.0,0.0,0.0,0.0,NaN


In [60]:
df_tweet_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              652 non-null    float64
 1   text            652 non-null    object 
 2   user_name       652 non-null    object 
 3   created_at      652 non-null    object 
 4   location        652 non-null    object 
 5   timestamp_ms    652 non-null    float64
 6   quote_count     652 non-null    float64
 7   reply_count     652 non-null    float64
 8   retweet_count   652 non-null    float64
 9   favorite_count  652 non-null    float64
 10  hashtags        336 non-null    object 
dtypes: float64(6), object(5)
memory usage: 56.2+ KB
